# Notes on Using Docker with Icepack
---

I have several requirements for running icepack within docker:
- It should run as seamlessly as possible on my existing network of servers, with full access all disk volumes,
- It should easily be able to run jupyter notebooks that are easily opened on the host system,
- It should have all of the necessary packages installed
- It should allow xapps to open on the host (e.g., emacs)
- The user id should match that on the host system to allow read/write to all disks
- It should work with /usr/bin/tcsh (my preference, change to bash or other shell as desired).

## Step 1: Setup Docker Container 

For this step, we defined how to setup the container with a file called This file should be named, `Dockerfile`. This step is like installing a new system. This is the place to install the necessary packages. If they are `pip` or `apt` installed from an active container, they will no be saved to the container image, and thus, will not be accessible the next time the container is run.

```
RUN sudo apt  -y install patchelf
RUN sudo apt -y install gdal-bin libgdal-dev
# install icepack stuff
RUN source /home/firedrake/firedrake/bin/activate && \
    pip install git+https://github.com/icepack/Trilinos.git && \
    pip install git+https://github.com/icepack/pyrol.git && \
    git clone https://github.com/icepack/icepack.git && \
    pip install --editable ./icepack && \
    pip install jupyter lab && \
    pip install siphash24
#
# DockerRequirements.txt contains all extra packages and is copied to containter
COPY  DockerRequirements.txt .
# Set up the kernel and other packages
RUN source /home/firedrake/firedrake/bin/activate && \
   pip install -r  DockerRequirements.txt && \
   python -m ipykernel install --user --name=firedrakeDocker
#
# Install x11-apps
RUN sudo apt -y install emacs tcsh
RUN sudo apt -y install x11-apps
# Configure X11 authentication and clean up apt cache
RUN sudo xauth && sudo rm -rf /var/lib/apt/lists/*
# Set timezone to LA
ENV TZ=America/Los_Angeles
RUN sudo ln -snf /usr/share/zoneinfo/$TZ /etc/localtime && \
    echo $TZ | sudo tee /etc/timezone
# Point jupyter config to a custom file (allows port range to changed)
ENV JUPYTER_CONFIG_DIR=~/.jupyter/config_icepack
#
#
# Set up with my user ID and shell preference and allow sudo 
RUN sudo useradd -m -u 3707 ian && echo "ian ALL=(ALL) NOPASSWD:ALL" | sudo tee /etc/sudoers.d/ian
RUN sudo usermod --shell /bin/tcsh ian
RUN sudo usermod -aG sudo ian
RUN  sudo chown -R  ian:ian  /home/firedrake
# Link the firedrake to my home directory
RUN sudo ln -s  /home/firedrake /firedrake
USER ian
# Use docker exec to start shells, so just hold
CMD sleep infinity

To build the container, run the following command in the same directory as `dockerFile`

`sudo docker build --tag dockertest1 .`

Note this sets up the kernel for Jupyter as `firedrakeDocker`.

## Step 2: Setup Docker Compose

The command `docker-compose` allows us to predefine many of the parameters that would have to be passed via `docker run`. It can be setup with a file `docker-compose.yml` as shown here:

```
services: # Defines the services (containers) that will run in your application
  dockertest:
    image: dockertest1  # Specifies the docker image to use
    container_name: icepack # Specifies the container name
    hostname: icepack   # This will make it look like a computer called icepack
    user: "ian" # Have it start up as me
    stdin_open: true # Keeps stdin open, allowing interaction with the container 
    tty: true # Allocates a pseudo-TTY, which is helpful for running interactive applications
    ports:
      - 9038-9088:9038-9088  # Maps this port range to the correpsponding range on the host
    volumes:  # Mounts volumes on the servers to the same mount points on the container
      - /tmp/.X11-unix:/tmp/.X11-unix
      - /home/ian:/home/ian
      - /Users/ian:/Users/ian      
      - /Volumes/insar1/ian:/Volumes/insar1/ian
      - /Volumes/insar3/ian:/Volumes/insar3/ian
      - /Volumes/insar4/ian:/Volumes/insar4/ian
      - /Volumes/insar5/ian:/Volumes/insar5/ian
      - /Volumes/insar6/ian:/Volumes/insar6/ian
      - /Volumes/insar7/ian:/Volumes/insar7/ian
      - /Volumes/insar8/ian:/Volumes/insar8/ian
      - /Volumes/insar9/ian:/Volumes/insar9/ian
      - /Volumes/insar10/ian:/Volumes/insar10/ian
      - /Volumes/insar11/ian:/Volumes/insar11/ian  

## Step 3: Run the Container

In this step `docker-compose` is used to run the container, with the commmand

`docker-compose up --remove-orphans`

This will start container up in a window and then pause. **If it fails to start or starts with an error**, try `docker remove`, which will prompt you remove any old container (this will not affect the dockerImage, only the prior container instance).

To pause the container, use:

`docker-compose stop`

The container can be restart with the same state with:

`docker-compose start`

To gracefully shut the container down use: 

`docker-compose down`

Or if its unresponsive, use:

`docker-compose kill`

## Run a Shell in the Container

Now to run a shell on the containter, 

`xhost + local:docker; docker exec -it -e DISPLAY=$DISPLAY -e XAUTHORITY=~/.Xauthority icepack /bin/tcsh`

The first part, `xhost + local:docker` should only need to be run the first time, but it doesn't hurt to run it each time. It allows the container to open xwindows on the host. The `docker exec` runs the last argmuent as a command on the host, which in this example is `tcsh`.  The `-it` says make it interactive, so you can work in the shell. The `-e` flag is to set environment variables on the host. The container name in this example is `icepack` as defined in the `docker-compose.yml` file above.

A nice feature of using shells this way is its easy to run multiple shells on the same container. And unlike docker-run, if you type `exit`, you only exit that one shell, not the whole container.

Note the way this is configured, anything saved on the `host` file systems mounted in the container will persist when the container shuts down. But anything written to the container will be lost, include software installs. If something needs to be `pip` installed or `apt` installed, verify the command while the container is running, and then add the command to the `Dockerfile` and rebuild the image for next time. 

There is a way to define a "local volume" for the container, but that just stores the somewhere deeps in /var and still takes up space on the host. So in most instances, its cleaner to mount the local files sytems as in this example. 

# Using Jupyter

Jupyter can be used, from the container and run in a browser on the host, but some care needs to taken in setting this up.  By default, Jupyter starts new sessions at port 8888 or some larger value if that value is taken. This is a problem if the host and the container both try to use the same port. Several steps are needed to fix this. 

### Allocate a Different Port Range for the Container

A different range of ports is allocated in the `docker-compose.yml` file described above with the line:

```
  ports:
      - 9038-9088:9038-9088  # Maps this port range to the correpsponding range on the host

### Configure Jupyter to Use the New Range

A cumbersome approach is to specify the port range on the command line when jupyter is started, but this requires the user to manage the assignment in situations with multiple instances of Jupyter. The port range can be specified in the jupyter config file, but that adds another problem if the host and container are using a shared home directory as described above.  To get around this, the Dockerfile has the command:

```ENV JUPYTER_CONFIG_DIR=~/.jupyter/config_icepack```

This means the container will now use a different config file than the host. The `jupyter_notebook_config.py` in this directory should look like this:

```
c.NotebookApp.ip = '0.0.0.0'  # Sets the notebook up for access by the host
c.NotebookApp.allow_remote_access = True  # Sets the notebook up for access by the host
c.NotebookApp.port = 9038  # This port starts just after the port range (will be increment by one with each new instance).

With the jupyter configure this way, jupyter can started with the simple command:

`jupyter lab`

Unlike on the host, this will not automatically start up the browser. Instead, it will print out an IP that can be opened in the host browser.** Make sure to use the one with 127.0.0.1 and not the one with the host name **

The IP can also be obtained on the host side with the command

`jupyter lab list | sed s/icepack/localhost/`

which outputs something like,

```
Currently running servers:
http://icepack:9038/?token=xxxx :: /home/ian/docker

Unfortunately this lists the hostname (icepack in this case). This can be fixed with the command:

`jupyter lab list | sed s/icepack/localhost/`

Its easy just to add an alias to your login to shorten this.

`alias jlist 'jupyter lab list | sed s/icepack/localhost/'`